In [2]:
from pprint import pprint
from util import generate_problem, verify_witness
INF = 1e9

In [3]:
def discrete_graph(N):
    mat = [[INF for _ in range(N)] for _ in range(N)]
    for i in range(N):
        mat[i][i] = 0
    return mat

def generate_d_graph(graph):
    N = len(graph)
    E = discrete_graph(N)
    
    for i in range(N):
        E[i][i] = 0 # redundant

    for i in range(N):
        for j in range(N):
            E[i][j] = graph[i][j]

    for k in range(N):
        for i in range(N):
            for j in range(N):
                E[i][j] = min(
                    E[i][j],
                    E[i][k] + E[k][j],
                )

    return E

def consistent(d_graph):
    for i in range(len(d_graph)):
        if d_graph[i][i] < 0: return False
    return True

def get_min_sol(d_graph):
    return [-d_graph[i][0] for i in range(len(d_graph))]

In [4]:
def intersect(I, J):
    """
    intersect intervals given by lists I and J
    note: both lists have to contain ordered disjoint intervals.
    """
    
    # TODO fix this merge lol, experimental
    return cleanup_intervals([
        [max(first[0], second[0]), min(first[1], second[1])] 
          for first in I for second in J  if max(first[0], second[0]) <= min(first[1], second[1])
    ])
    
def compose(I, J):
    """
    compose lists of intervals I and J
    note: make sure they are ordered and disjoint
    """

    return cleanup_intervals([
        (i[0] + j[0], i[1] + j[1])
        for i in I for j in J
    ])

def cleanup_intervals(I):
    """
    given a list of intervals, produce equivalent ordered disjoint intervals
    """
    
    ret = []
    
    for begin, end in sorted(I):
        if ret and ret[-1][1] >= begin - 1:
            ret[-1][1] = max(ret[-1][1], end)
        else:
            ret.append( [begin, end] )
    
    return [(a, b) for a, b in ret]

    
def PC_1(T):
    # i need constraints to be in a specific form to perform random access T_ij
    num_variables = max([max(t['i'], t['j']) for t in T]) + 1
    mat = [ [ None for i in range(num_variables) ] for j in range(num_variables) ]
    for constr in T:
        i, j, intervals = constr['i'], constr['j'], constr['intervals']
        
        mat[i][j] = intervals
        mat[j][i] = [(-b, -a) for a, b in reversed(intervals)]
        # for reverse, we take each interval (a,b) and convert it into (-b,-a)
        # note we want to also reverse the order of all intervals so that it is again increasing
        
    
    # actual PC-1 core algorithm
    
    changed = True
    while changed:
        changed = False
        for k in range(num_variables):
            for i in range(num_variables):
                for j in range(num_variables):
                    # T_ij = T_ij + T_ik x T_kj

                    if mat[i][k] == None or mat[k][j] == None:
                        # if we cannot compose
                        continue

                    elif mat[i][j] == None:
                        # if unset it means no constraint, so only compose
                        mat[i][j] = compose(mat[i][k], mat[k][j])

                    else: # full intersect&compose
                        op = intersect(mat[i][j], compose(mat[i][k], mat[k][j]))
                        if mat[i][j] != op:
                            mat[i][j] = op
                            changed = True

                    if not mat[i][j]:
                        print('PC-1: unsat')
                        return
    
    # convert back to constraint form
    S = []
    for i in range(num_variables):
        for j in range(i, num_variables):
            if mat[i][j]:
                S += [{'i': i, 'j': j, 'intervals': mat[i][j]}]
    return S

In [14]:
num_variables = 15
T = generate_problem(
    variables=num_variables,
    constraint_probability=0.1,
)
S = PC_1(T)
S

[{'i': 0, 'j': 0, 'intervals': [(-100, 100)]},
 {'i': 0, 'j': 1, 'intervals': [(-100, 100)]},
 {'i': 0, 'j': 2, 'intervals': [(-100, 100)]},
 {'i': 0, 'j': 3, 'intervals': [(-100, 100)]},
 {'i': 0, 'j': 4, 'intervals': [(-100, 100)]},
 {'i': 0, 'j': 5, 'intervals': [(-100, 100)]},
 {'i': 0, 'j': 6, 'intervals': [(-100, 100)]},
 {'i': 0, 'j': 7, 'intervals': [(-100, 100)]},
 {'i': 0, 'j': 8, 'intervals': [(-100, 100)]},
 {'i': 0, 'j': 9, 'intervals': [(-100, 100)]},
 {'i': 0, 'j': 10, 'intervals': [(-100, 100)]},
 {'i': 0, 'j': 11, 'intervals': [(-100, 100)]},
 {'i': 0, 'j': 12, 'intervals': [(-100, 100)]},
 {'i': 0, 'j': 13, 'intervals': [(-100, 100)]},
 {'i': 0, 'j': 14, 'intervals': [(-100, 100)]},
 {'i': 1, 'j': 1, 'intervals': [(-140, 140)]},
 {'i': 1, 'j': 2, 'intervals': [(-200, 200)]},
 {'i': 1, 'j': 3, 'intervals': [(-200, 200)]},
 {'i': 1, 'j': 4, 'intervals': [(-200, 200)]},
 {'i': 1, 'j': 5, 'intervals': [(-93, -54), (-9, 47)]},
 {'i': 1, 'j': 6, 'intervals': [(-200, 200)]},